# Experiment 1

In [10]:
import sys
sys.path.append('/Users/kenzatazi/Documents/CDT/Code')

from load import era5, data_dir, value
from sklearn.preprocessing import MinMaxScaler, StandardScaler
from matplotlib import pyplot as plt
import pandas as pd
import numpy as np
import GPy
from tqdm import tqdm
import scipy as sp
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.model_selection import KFold
from mfdgp.utils.metrics import msll, r2_low_vs_high

In [3]:
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error, r2_score

In [4]:
import emukit
from emukit.model_wrappers.gpy_model_wrappers import GPyMultiOutputWrapper
from emukit.multi_fidelity.models import GPyLinearMultiFidelityModel
from emukit.multi_fidelity.convert_lists_to_array import convert_x_list_to_array, convert_xy_lists_to_arrays

## Prepare data

In [1]:
# Load data
minyear = 1980
maxyear = 2010

In [5]:
gauge_df = value.all_gauge_data(minyear, maxyear, monthly=True)
station_names = gauge_df.drop_duplicates('name')['name']

# Get CV scheme
cv_locs = np.load('/Users/kenzatazi/Documents/CDT/Code/mfdgp/experiments/exp1/cv/exp1_cv_locs.npy')
cv_locs = cv_locs.reshape(-1, 2)

station_list = []
for loc in cv_locs:
    station_row = gauge_df[(gauge_df['latitude'] == loc[1]) | (gauge_df['longitude'] == loc[0])].iloc[0]
    station_list.append(station_row['name'])
station_arr = np.array(station_list)

# Split indexes
kf = KFold(n_splits=5)

cv_train_list = []
cv_test_list = []

for train_index, test_index in kf.split(station_arr):
    hf_train, hf_test = station_arr[train_index], station_arr[test_index]
    cv_train_list.append(hf_train)
    cv_test_list.append(hf_test)

In [22]:
# Split data
cv_x_train_hf = []
cv_y_train_hf = []
cv_x_train_lf = []
cv_y_train_lf = []
cv_x_val = []
cv_y_val = []
lf_lambdas = []


for i in range(len(cv_train_list)):

    hf_train_list = []
    for station in cv_train_list[i]:
        station_ds = value.gauge_download(
            station, minyear=minyear, maxyear=maxyear)
        hf_train_list.append(station_ds.dropna().reset_index())
    hf_train_df = pd.concat(hf_train_list)

    val_list = []
    for station in cv_test_list[i]:
        station_ds = value.gauge_download(
            station, minyear=minyear, maxyear=maxyear)
        val_list.append(station_ds.dropna().reset_index())
    val_df = pd.concat(val_list)

    era5_df = era5.value_gauge_download(
        list(cv_test_list[i]) + list(cv_train_list[i]), minyear=minyear, maxyear=maxyear)
    
    hf_train_df.sort_values(by='time', inplace=True)
    val_df.sort_values(by='time', inplace=True)
    lf_train_df = era5_df.reset_index().sort_values(by='time')
    
    # Prepare data
    
    # Transformations
    lf_train_df['tp_tr'], lf_lambda = sp.stats.boxcox(
        lf_train_df['tp'].values + 0.01)
    hf_train_df['tp_tr'] = sp.stats.boxcox(
        hf_train_df['tp'].values + 0.01, lmbda=lf_lambda)
    val_df['tp_tr'] = sp.stats.boxcox(
        val_df['tp'].values + 0.01, lmbda=lf_lambda)

    # Splitting
    x_train_lf = lf_train_df[['time', 'lat', 'lon', 'z']].values.reshape(-1, 4)
    y_train_lf = lf_train_df['tp_tr'].values.reshape(-1, 1)
    x_train_hf = hf_train_df[['time', 'latitude', 'longitude', 'altitude']].values.reshape(-1, 4)
    y_train_hf = hf_train_df[['tp_tr']].values.reshape(-1, 1)
    x_val = val_df[['time', 'latitude', 'longitude', 'altitude']].values.reshape(-1, 4)
    y_val = val_df['tp_tr'].values.reshape(-1, 1)

    # Scaling
    scaler = StandardScaler().fit(x_train_hf)
    x_train_hf1 = scaler.transform(x_train_hf)
    x_train_lf1 = scaler.transform(x_train_lf)
    x_val1 = scaler.transform(x_val)
    
    cv_x_train_hf.append(x_train_hf1)
    cv_y_train_hf.append(y_train_hf)
    cv_x_train_lf.append(x_train_lf1)
    cv_y_train_lf.append(y_train_lf)
    cv_x_val.append(x_val1)
    cv_y_val.append(y_val)
    lf_lambdas.append(lf_lambda)

value
/Users/kenzatazi/Documents/CDT/Code/data/ERA5/combi_data_value_03-2023.csv
value
/Users/kenzatazi/Documents/CDT/Code/data/ERA5/combi_data_value_03-2023.csv
value
/Users/kenzatazi/Documents/CDT/Code/data/ERA5/combi_data_value_03-2023.csv
value
/Users/kenzatazi/Documents/CDT/Code/data/ERA5/combi_data_value_03-2023.csv
value
/Users/kenzatazi/Documents/CDT/Code/data/ERA5/combi_data_value_03-2023.csv


In [24]:
np.save('data/cv_x_train_hf_value_1980-2010.npy', cv_x_train_hf)
np.save('data/cv_y_train_hf_value_1980-2010.npy', cv_y_train_hf)
np.save('data/cv_x_train_lf_value_1980-2010.npy', cv_x_train_lf)
np.save('data/cv_y_train_lf_value_1980-2010.npy', cv_y_train_lf)
np.save('data/cv_y_val_value_1980-2010.npy', cv_y_val)
np.save('data/cv_x_val_value_1980-2010.npy', cv_x_val)
np.save('data/lf_lambda_1980-2010.npy', lf_lambdas)

 /Users/kenzatazi/miniconda3/envs/mfdgp/lib/python3.8/site-packages/numpy/lib/npyio.py:521: VisibleDeprecationWarning:Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.


## Load data

In [20]:
cv_x_train_hf = np.load('data/cv_x_train_hf_value_1980-2010.npy', allow_pickle=True)
cv_y_train_hf = np.load('data/cv_y_train_hf_value_1980-2010.npy', allow_pickle=True)
cv_x_train_lf = np.load('data/cv_x_train_lf_value_1980-2010.npy', allow_pickle=True)
cv_y_train_lf = np.load('data/cv_y_train_lf_value_1980-2010.npy', allow_pickle=True)
cv_x_val = np.load('data/cv_x_val_value_1980-2010.npy', allow_pickle=True)
cv_y_val = np.load('data/cv_y_val_value_1980-2010.npy', allow_pickle=True)
lf_lambdas = np.load('data/lf_lambda_1980-2010.npy', allow_pickle=True)

In [27]:
cv_x_train_lf[i][:tr*28]
tr


48

## MFDGP

In [29]:
R2_all = []

for t in tqdm([1, 2, 4, 8, 15, 30]):

    R2 = []

    for i in range(len(cv_train_list)):

        tr = 12 * t

        # Input data
        X_train, Y_train = convert_xy_lists_to_arrays([cv_x_train_lf[i][:tr*28], cv_x_train_hf[i][:tr*28]], [
                                                      cv_y_train_lf[i][:tr*28], cv_y_train_hf[i][:tr*28]])

        # Train and evaluate
        kern1 = GPy.kern.Matern52(input_dim=4, ARD=True)
        kernels = [kern1, GPy.kern.Matern52(input_dim=4, ARD=True)]
        lin_mf_kernel = emukit.multi_fidelity.kernels.LinearMultiFidelityKernel(
            kernels)
        gpy_lin_mf_model = GPyLinearMultiFidelityModel(
            X_train, Y_train, lin_mf_kernel, n_fidelities=2,)
        gpy_lin_mf_model.mixed_noise.Gaussian_noise.fix(0)
        gpy_lin_mf_model.mixed_noise.Gaussian_noise_1.fix(0)
        lin_mf_model = GPyMultiOutputWrapper(
            gpy_lin_mf_model, 2, n_optimization_restarts=5)
        lin_mf_model.optimize()

        # Load and prep test data
        x_val1, y_val = cv_x_val[i][:tr*7], cv_y_val[i][:tr*7]
        n = x_val1.shape[0]
        x_met = convert_x_list_to_array([x_val1, x_val1])

        # ALL
        y_pred0, y_var0 = lin_mf_model.predict(x_met[n:])
        y_pred_low0, y_var_low0 = lin_mf_model.predict(x_met[:n])

        y_pred = sp.special.inv_boxcox(y_pred0, lf_lambdas[i]).reshape(-1)
        y_true = sp.special.inv_boxcox(y_val, lf_lambdas[i]).reshape(-1)
        R2.append(r2_score(y_true, y_pred))

    print('Mean R2 = ', np.mean(R2), '±', np.std(R2))
    R2_all.append([np.mean(R2), np.std(R2)])

np.savetxt('app_c_value.npy', R2_all)

  0%|          | 0/6 [00:00<?, ?it/s]

Optimization restart 1/5, f = 668.4692951103332
Optimization restart 2/5, f = 668.4692958640181
Optimization restart 3/5, f = 668.469297625448
Optimization restart 4/5, f = 668.4692957384622


  0%|          | 0/6 [10:56<?, ?it/s]

KeyboardInterrupt caught, calling on_optimization_end() to round things up


KeyboardInterrupt: 

In [14]:
plt.figure()
R2_arr = np.array(R2_all)
plt.errorbar([0, 2, 4, 8, 15, 30], R2_arr[:, 0], yerr=R2_arr[:, 1],
             capsize=5, marker='.', linestyle='none')
reg = LinearRegression().fit(
    np.array([0, 2, 4, 8, 15, 30]).reshape(-1, 1), R2_arr[:, 0])
y = reg.predict(np.linspace(0, 30, 10).reshape(-1, 1))
plt.plot(np.linspace(0, 30, 10).reshape(-1, 1), y, linestyle='--')
plt.ylabel('$R^2$')
plt.xlabel('Years')
plt.savefig('appendix_C_plot.png', dpi=300)


Mean RMSE =  1.8554768746561332 ± 0.0
Mean R2 =  0.4456420375310417 ± 0.0
5th RMSE =  0.9196423098694533 ± 0.0
95th RMSE =  5.077877083218228 ± 0.0
MSLL=  1.0922637325560496 ± 0.0
Mean RMSE =  1.9670327346686514 ± 0.0
Mean R2 =  0.3769794489376289 ± 0.0
5th RMSE =  1.0649328671758052 ± 0.0
95th RMSE =  4.168218379910711 ± 0.0
MSLL=  27390097.713403944 ± 0.0
